In [1]:
## load packages
## if you haven't installed those packages yet (for example, if you don't have pandas), please type in your terminal: pip install pandas
import pandas as pd
from bs4 import BeautifulSoup
import requests
import os
import csv

In [6]:
# please put this script and 'List of companies.xlsx'(your file) in the same directory
directory = os.getcwd() 
file_name = 'List of companies.xlsx'
path = os.path.join(directory,file_name)
df = pd.read_excel(path,header= 0)
cik = [df['cik'][i] for i in range(len(df))] # get all the cik

In [90]:
url_first = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK='
url_last = '&type=def+14&dateb=&owner=exclude&count=40'
url = url_first+str(cik[1])+url_last

In [139]:
year_range = [2013,2014,2015,2016] ## the range you need to specify

In [158]:
def get_url1(obj):
    table = obj.find_all('table',class_ = 'tableFile2')
    date_valid = []
    url_valid = []
    if table[0].a==None:
        return False,False
    else:
        doc_length = len(table[0].find_all('tr'))
        for i in range(1,doc_length):
            date = table[0].find_all('tr')[i].find_all('td')[3].getText()
            year = date.strip().split('-')[0]
            #print year
            if int(year) in year_range:
                date_valid.append(date)
                url_valid.append(table[0].find_all('tr')[i].a['href'])
            else:
                pass
    return date_valid,url_valid

In [159]:
def get_url2(obj):
    table = obj.find_all('table',class_ = 'tableFile')
    if table[0].a==None:
        return False
    else:
        return table[0].a['href']

In [168]:
url_first = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK='
url_last = '&type=def+14&dateb=&owner=exclude&count=40'
url_prefix = 'https://www.sec.gov'

cik_archives = [] ## store the cik which has file associated
date_archives = [] ## the url for "Document" Button
web2_archives = [] ## the url for the hml file
for i in range(len(df)):
    url = url_first+(str(cik[i]))+url_last
    #print url
    data = requests.get(url).content
    soup = BeautifulSoup(data, 'html.parser')
    date,web = get_url1(soup)
    if date == False:
        continue
    url2_list = [url_prefix + url.encode('utf-8') for url in web]
    #url2 = url_prefix + web.encode('utf-8')
    url3_list = []
    list_length = len(url2_list)
    for j in range(list_length):
        data2 = requests.get(url2_list[j]).content
        soup2 = BeautifulSoup(data2,'html.parser')
        web2 = get_url2(soup2)
        if web2 == False:
            continue
        url3 = url_prefix + web2.encode('utf-8')
        #url3_list.append(url3)
        web2_archives.append(url3)
        date_archives.append(date[j])
        cik_archives.append(cik[i])         
        

In [169]:
## build a pandas format
web_url = pd.DataFrame(
    {'cik': cik_archives,
    'link': web2_archives,
    'date': date_archives
    })

In [171]:
## write the data to a csv file
## note this file is in the same directory of this script
web_url.to_csv('web_url.csv',index = False)

In [170]:
## look and check
web_url

,cik,date,link
0,1394954,2014-04-22,https://www.sec.gov/Archives/edgar/data/139495...
1,1394954,2013-04-26,https://www.sec.gov/Archives/edgar/data/139495...
2,1499912,2016-04-26,https://www.sec.gov/Archives/edgar/data/149991...
3,1499912,2015-08-04,https://www.sec.gov/Archives/edgar/data/149991...
4,1499912,2015-05-01,https://www.sec.gov/Archives/edgar/data/149991...
5,1499912,2014-04-30,https://www.sec.gov/Archives/edgar/data/149991...
6,1499912,2013-04-30,https://www.sec.gov/Archives/edgar/data/149991...
7,1023024,2016-04-14,https://www.sec.gov/Archives/edgar/data/102302...
8,1023024,2015-04-24,https://www.sec.gov/Archives/edgar/data/102302...
9,1023024,2014-04-11,https://www.sec.gov/Archives/edgar/data/102302...
